In [ ]:
import kagglehub

In [ ]:
path = kagglehub.dataset_download("vinayakshanawad/cement-manufacturing-concrete-dataset")


In [ ]:
print("Path to dataset files:", path)


In [ ]:
path = "/data/ephemeral/home/.cache/kagglehub/datasets/vinayakshanawad/cement-manufacturing-concrete-dataset/versions/1"

In [ ]:
import fireducks.pandas as pd
df = pd.read_csv(path + "/concrete.csv")

df.head()

In [ ]:
df.describe()

In [ ]:
df.corr()


In [ ]:
import torch

In [ ]:
import sklearn

In [ ]:
train,test = sklearn.model_selection.train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
class ConcreteDataset(torch.utils.data.Dataset):
    def __init__(self, df, mean,std):
        self.df = df
        self.mean = mean
        self.std = std


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = self.df.iloc[idx][['cement','slag','ash','water','superplastic','coarseagg','fineagg','age']].to_numpy()
        y = self.df.iloc[idx][['strength']].to_numpy()
        x = torch.tensor(x, dtype=torch.float32)    
        y = torch.tensor(y, dtype=torch.float32)
        x = (x - self.mean[:-1]) / self.std[:-1]
        # y = (y - self.mean[7]) / self.std[7]
        y = torch.log(y)
        # if self.transform:
        #     x = self.transform(x)
            
        return x, y

In [ ]:
# dataset = ConcreteDataset(df)

# dataset[0]

In [ ]:
train

In [ ]:
mean = train[['cement','slag','ash','water','superplastic','coarseagg','fineagg','age','strength']].to_numpy().mean(axis=0)
std = train[['cement','slag','ash','water','superplastic','coarseagg','fineagg','age','strength']].to_numpy().std(axis=0)
mean = torch.tensor(mean, dtype=torch.float32)
std = torch.tensor(std, dtype=torch.float32)

In [ ]:
# import torchvision.transforms as transforms
# transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
dataset_train = ConcreteDataset(train,mean,std)
dataset_test = ConcreteDataset(test,mean,std)


In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset_train, batch_size=100, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=100, shuffle=False)

In [ ]:
class simple_nn(torch.nn.Module):
    def __init__(self):
        super(simple_nn, self).__init__()
        self.fc1 = torch.nn.Linear(8, 16)
        self.fc2 = torch.nn.Linear(16, 32)
        self.fc3 = torch.nn.Linear(32, 1)
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        xf = self.fc3(x)
        return xf,x


In [ ]:
model = simple_nn()
model.cuda()

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import numpy as np
losses = []
val_losses = []
errors = []
for i in range(200):
    Lo = []
    for inputs, outputs in train_loader:
        inputs = inputs.cuda()
        outputs = outputs.cuda()
        model.zero_grad()
        pred,_ = model(inputs)
        loss = torch.nn.MSELoss()(pred, outputs)
        
        loss.backward()
        optimizer.step()
        Lo.append(loss.item())
    
    losses.append(np.mean(Lo))
    err = []
    vLo = []
    for inputs, outputs in test_loader:
        inputs = inputs.cuda()
        outputs = outputs.cuda()
        pred,_ = model(inputs)
        loss = torch.nn.MSELoss()(pred, outputs)
        vLo.append(loss.item())
        err.append(abs(torch.exp(pred.detach().cpu())- torch.exp(outputs.detach().cpu())).numpy())
    errors.append(np.concatenate(err,axis=0).mean())
    val_losses.append(np.mean(vLo))
    print(f'losses: {np.mean(losses)}')
    print(f'val_losses: {np.mean(val_losses)}')
    print(f'errors: {np.concatenate(err,axis=0).mean()}')

In [ ]:
len(val_losses)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.plot(val_losses)
plt.show()


In [ ]:
# np.concatenate(errors,axis=0).shape

In [ ]:
np.array(errors).shape

In [ ]:
plt.plot(np.array(errors))
plt.plot(len(errors)*[np.mean(df['strength'].to_numpy())])
plt.plot(len(errors)*[np.mean(df['strength'].to_numpy())+np.std(df['strength'].to_numpy())])
plt.plot(len(errors)*[np.mean(df['strength'].to_numpy())-np.std(df['strength'].to_numpy())])
plt.show()

In [ ]:
# plt.scatter(pred_y, residuals, alpha=0.7)


In [ ]:
mean

In [ ]:

err = []
mean_err = []
pred_y = []

for inputs, outputs in test_loader:
    inputs = inputs.cuda()
    outputs = outputs.cuda()
    pred,_ = model(inputs)
    loss = torch.nn.MSELoss()(pred, outputs)
    val_losses.append(loss.item())
    pred_y.append(torch.exp(pred.detach().cpu()).numpy())
    y_p = (torch.exp(pred.detach().cpu())- torch.exp(outputs.detach().cpu()))**2
    y_t = (mean[-1].repeat(outputs.shape[0]).reshape(outputs.shape[0],1) - torch.exp(outputs.detach().cpu()))**2
    err.append(y_p.numpy())
    mean_err.append(y_t.numpy())


In [ ]:
test[['cement','slag','ash','water','superplastic','coarseagg','fineagg','age']].to_numpy().shape

In [ ]:
err = np.concatenate(err,axis=0)
mean_err = np.concatenate(mean_err,axis=0)


In [ ]:
pred_y = np.concatenate(pred_y,axis=0)

In [ ]:
print(err.shape)
print(mean_err.shape)
print(pred_y.shape)

In [ ]:
1030

In [ ]:
print(1- np.mean(err)/np.mean(mean_err))

In [ ]:
n = 1030
p = 8

In [ ]:
print(1- (np.mean(err)/(n-p-1))/(np.mean(mean_err)/(n-1)))

In [ ]:
max(err)

In [ ]:
err[np.where(err < 100)[1]].shape

In [ ]:
err[np.where(err < 300)[1]].shape

In [ ]:
plt.scatter(pred_y,err, alpha=0.7)


In [ ]:
# !pip install statsmodels

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
import statsmodels.api as sm


In [ ]:
err.flatten().shape

In [ ]:
X_with_const = sm.add_constant(test[['cement','slag','ash','water','superplastic','coarseagg','fineagg','age']].to_numpy())  # 상수항 추가
bp_test = het_breuschpagan(err.flatten(), X_with_const)


In [ ]:
print("Breusch-Pagan Test Results:")
print(f"Lagrange Multiplier Statistic: {bp_test[0]}")
print(f"p-value: {bp_test[1]}")
print(f"F-statistic: {bp_test[2]}")
print(f"F-statistic p-value: {bp_test[3]}")


In [ ]:
if bp_test[1] < 0.05:
    print("The assumption of homoscedasticity is violated.")
else:
    print("The assumption of homoscedasticity is satisfied.")


In [ ]:
# df['strength'].to_numpy()
plt.hist(df['strength'].to_numpy(),bins=100)

In [ ]:
feature_list = []
for inputs, outputs in test_loader:
    inputs = inputs.cuda()
    outputs = outputs.cuda()
    pred,features = model(inputs)
    features = features.detach().cpu().numpy()
    feature_list.extend(features)
feature_list = np.array(feature_list)

In [ ]:
feature_list.shape

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
tsne.fit_transform(feature_list)
plt.scatter(tsne.embedding_[:, 0], tsne.embedding_[:, 1], c=test['strength'].to_numpy())
plt.show()

In [ ]:
feature_list = []
for inputs, outputs in train_loader:
    inputs = inputs.cuda()
    outputs = outputs.cuda()
    pred,features = model(inputs)
    features = features.detach().cpu().numpy()
    feature_list.extend(features)
feature_list = np.array(feature_list)
tsne.fit_transform(feature_list)
plt.scatter(tsne.embedding_[:, 0], tsne.embedding_[:, 1], c=train['strength'].to_numpy())
plt.show()

In [ ]:
model

In [ ]:
mean[:-1]

In [ ]:
std[:-1]

In [ ]:
import tqdm

In [ ]:
flag = 0
save_SSE = []
save_SST = []
patience = 10
lr_decay_factor = 0.5  
for x,y in tqdm.tqdm(test_loader):
    x = x.cuda()
    y = y.cuda()
    # print(x.shape)
    # brewa
    y_opt = y.detach()
    init_x = torch.randn(x.shape[0],x.shape[1], device='cuda', requires_grad=True) # .requires_grad_(True).cuda()
    # init_x = mean[:-1].expand(10,8).clone().requires_grad_(True)
    init_x.requires_grad = True
    optimizer = optim.Adam([init_x], lr=0.1)
    
    
    for param in model.parameters():
        param.requires_grad = False

    min_val = 1e6
    min_yp = None
    min_yt = None

    no_improve_epochs = 0

    for i in range(10000):
        optimizer.zero_grad()
        # with torch.no_grad():
        pred,features = model(init_x)
        loss = torch.nn.MSELoss()(pred, y_opt)
        loss.backward()
        optimizer.step()
        if loss.item() < min_val:
            min_val = loss.item()
            min_yp = (init_x.detach().cpu()*std[:-1] - x.detach().cpu()*std[:-1])**2
            min_yt = (mean[:-1].repeat(init_x.shape[0]).reshape(init_x.shape[0],-1) - x.detach().cpu()*std[:-1])**2
        
            # print(min_x.mean())
        else:
            no_improve_epochs += 1
        
        if no_improve_epochs > patience:
            optimizer.param_groups[0]['lr'] *= lr_decay_factor
            # print(f'lr decayed to {optimizer.param_groups[0]["lr"]}')
            no_improve_epochs = 0
        # print((init_x.detach().cpu() - x.cpu()).numpy().mean())
    
    save_SSE.append(min_yp.numpy())
    save_SST.append(min_yt.numpy())
    


In [ ]:
save_SSE = np.concatenate(save_SSE,axis=0)
save_SST = np.concatenate(save_SST,axis=0)
print(save_SSE.shape)
print(save_SST.shape)
print(1- np.mean(save_SSE)/np.mean(save_SST))


In [ ]:
# save = torch.tensor(np.array(save))*std[7] + mean[7]


In [ ]:
std# np.concatenate(save1,axis=0).shape

In [ ]:
save_SSE.shape

In [ ]:
save_SSE.mean(axis=1).shape

In [ ]:
save_SST.mean(axis=1).shape

In [ ]:
# save_SSE.mean(axis=1).shape

In [ ]:
df.columns

In [ ]:
1 - (save_SSE.mean(axis=0) / save_SST.mean(axis=0))

In [ ]:
df.columns


In [ ]:
plt.hist(save_SSE.mean(axis=0) / save_SST.mean(axis=0))

In [ ]:
# plt.hist(save1.mean(axis=1),bins=100)


In [ ]:
std

In [ ]:
mean